In [1]:
import matplotlib.pyplot as plt
import pandas
import pandas as pd
import seaborn as sns
import json
import os
import deepsig
from IPython.display import display

2023-08-07 18:28:24.673557: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-07 18:28:24.731335: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-07 18:28:24.732236: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-07 18:28:26.142389: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def convert_to_json_of_arrays(array_of_jsons):
    json_of_arrays = {}

    # Iterate through each JSON in the array
    for json_obj in array_of_jsons:
        for key, value in json_obj.items():
            # Check if the key exists in the new JSON, if not, create an array
            if key not in json_of_arrays:
                json_of_arrays[key] = []

            # Append the value to the corresponding array
            json_of_arrays[key].append(value)

    return json_of_arrays

In [3]:
def read_csv_files_from_folder(folder_path):
    # Initialize an empty list to store DataFrames
    dfs = []

    # Get a list of files in the folder
    file_list = os.listdir(folder_path)

    # Iterate through the files in the folder
    for file_name in file_list:
        # Check if the file has a .csv extension
        if file_name.endswith(".csv"):
            file_path = os.path.join(folder_path, file_name)

            # Read the CSV file into a DataFrame
            df = pd.read_csv(file_path)

            # Append the DataFrame to the list
            dfs.append(df)

    # Concatenate all DataFrames into a single DataFrame
    combined_df = pd.concat(dfs, ignore_index=True)

    return combined_df

In [4]:
def read_json_files_from_folder(folder_path):
    json_list = []

    # Get a list of files in the folder
    file_list = os.listdir(folder_path)

    # Iterate through the files in the folder
    for file_name in file_list:
        # Check if the file has a .json extension
        if file_name.endswith(".json"):
            file_path = os.path.join(folder_path, file_name)

            # Read the contents of the JSON file
            with open(file_path, "r") as file:
                try:
                    # Parse the JSON data and append it to the list
                    json_data = json.load(file)
                    json_list.append(json_data)
                except json.JSONDecodeError:
                    # Handle parsing errors, if any
                    print(f"Error parsing JSON from file: {file_name}")

    return json_list

In [5]:
cols = ['dataset', 'method', 'fitness_rule', 'fitness', 'ACC', 'MCC', 'f1_score', 'avg_odds_diff', 'stat_par_diff', 'eq_opp_diff']

In [6]:
results = read_csv_files_from_folder('results')
results.replace({'ftl_mlp_initializer': 'Fair Transition Loss', 'adversarial_debiasing_initializer': 'Adversarial Debiasing', 'gerry_fair_classifier_initializer': 'Gerry Fair Classifier', 'prejudice_remover_initializer': 'Prejudice Remover', 'simple_mlp_initializer': 'Standard MLP (baseline)'}, inplace=True)
results.replace({'adult_dataset_reader': 'Adult Income', 'compas_dataset_reader': 'Compas Recidivism', 'german_dataset_reader': 'German Credit'}, inplace=True)
results.rename(columns={'avg_odds_diff': 'Equalized Odds', 'stat_par_diff': 'Statistical Parity', 'eq_opp_diff': 'Equal Opportunity', 'MCC': 'Mathew Correlation', 'ACC': 'Accuracy'}, inplace=True)

In [7]:
fitness_rules_target_metrics = {
    'mcc_parity': {'performance': 'Mathew Correlation', 'fairness': 'Statistical Parity'},
    'mcc_opportunity': {'performance': 'Mathew Correlation', 'fairness': 'Equal Opportunity'},
    'mcc_odds': {'performance': 'Mathew Correlation', 'fairness': 'Equalized Odds'},
    'acc_parity': {'performance': 'Accuracy', 'fairness': 'Statistical Parity'},
    'acc_opportunity': {'performance': 'Accuracy', 'fairness': 'Equal Opportunity'},
    'acc_odds': {'performance': 'Accuracy', 'fairness': 'Equalized Odds'}
}

In [8]:
display(results)

,Unnamed: 0,overall_acc,bal_acc,Equalized Odds,disp_imp,Statistical Parity,Equal Opportunity,theil_ind,f1_score,TPR,...,FOR,Accuracy,Mathew Correlation,fitness,solution,tune_results_history,fitness_rule,method,dataset,best_solution_tf_history
0,0,0.650000,0.477941,0.022839,1.007030,0.006558,0.028241,0.087680,0.804954,0.955882,...,0.000000,0.650000,0.000000,-0.006558,{'dropout': 0.18124389809074126},"[{'overall_acc': 0.59375, 'bal_acc': 0.4481132...",mcc_parity,Standard MLP (baseline),German Credit,"{'loss': [0.7094872196515402, 0.64226885371738..."
1,1,0.645000,0.484962,0.073451,1.064832,0.059217,0.045977,0.077539,0.806250,0.969925,...,0.000000,0.645000,0.000000,-0.059217,"{'dropout': 0.022732599406043996, 'privileged_...","[{'overall_acc': 0.00625, 'bal_acc': 0.0043859...",mcc_parity,Fair Transition Loss,German Credit,"{'loss': [0.8503885070482889, 0.78124442100524..."
2,2,0.740000,0.659133,0.025781,0.954128,0.038462,0.017416,0.092947,0.823129,0.937984,...,0.228571,0.740000,0.400810,0.362348,{'adversary_loss_weight': 0.6148404367300289},"[{'overall_acc': 0.70625, 'bal_acc': 0.5580357...",mcc_parity,Adversarial Debiasing,German Credit,NaN
3,3,0.545000,0.397810,0.070884,0.895425,0.077859,0.056402,0.199996,0.775801,0.795620,...,0.000000,0.545000,0.000000,-0.077859,{'eta': 49.471477392162484},"[{'overall_acc': 0.56875, 'bal_acc': 0.4136363...",mcc_parity,Prejudice Remover,German Credit,NaN
4,4,0.615000,0.448905,0.066751,0.894180,0.087912,0.050342,0.121159,0.828283,0.897810,...,0.000000,0.615000,0.000000,-0.066751,{'dropout': 0.0037868792501197348},"[{'overall_acc': 0.56875, 'bal_acc': 0.4099099...",mcc_odds,Standard MLP (baseline),German Credit,"{'loss': [0.7087434199121263, 0.61068538427352..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1662,25,0.852736,0.776522,0.032846,0.388373,0.160795,0.001531,0.115084,0.673689,0.629003,...,0.113442,0.852736,0.581666,0.851205,{'dropout': 0.0043869367282519224},"[{'overall_acc': 0.8454947484798231, 'bal_acc'...",acc_opportunity,Standard MLP (baseline),Adult Income,"{'loss': [0.38556268092415, 0.3363742627719026..."
1663,26,0.837479,0.782958,0.050428,0.424477,0.176829,0.024213,0.111816,0.672460,0.675168,...,0.106890,0.837479,0.564398,0.813266,"{'dropout': 0.18094438096604942, 'privileged_d...","[{'overall_acc': 0.7994748479823107, 'bal_acc'...",acc_opportunity,Fair Transition Loss,Adult Income,"{'loss': [0.43217717875093253, 0.3988633977352..."
1664,27,0.848977,0.762294,0.051293,0.441320,0.134065,0.056000,0.125257,0.658841,0.591215,...,0.125413,0.848977,0.569623,0.792977,{'adversary_loss_weight': 0.0006190798728882424},"[{'overall_acc': 0.8188225538971807, 'bal_acc'...",acc_opportunity,Adversarial Debiasing,Adult Income,NaN
1665,28,0.845550,0.764002,0.027326,0.417448,0.145355,0.002915,0.129761,0.666189,0.594710,...,0.131871,0.845550,0.574608,0.842635,{'eta': 0.43671873092617697},"[{'overall_acc': 0.8452183526810393, 'bal_acc'...",acc_opportunity,Prejudice Remover,Adult Income,NaN


In [10]:
datasets = results['dataset'].unique().tolist()
datasets

['German Credit', 'bank_dataset_reader', 'Adult Income', 'Compas Recidivism']

In [11]:
fitness_rules = results['fitness_rule'].unique().tolist()
fitness_rules

['mcc_parity',
 'mcc_odds',
 'mcc_opportunity',
 'acc_parity',
 'acc_odds',
 'acc_opportunity']

In [12]:
methods = results['method'].unique().tolist()
methods

['Standard MLP (baseline)',
 'Fair Transition Loss',
 'Adversarial Debiasing',
 'Prejudice Remover',
 'Gerry Fair Classifier']

In [23]:
multi_aso_data_list = []
for d in datasets:
    multi_aso_data = []
    for f in fitness_rules:
        methods_results = []
        for m in methods:
            r = results.loc[ (results['dataset'] == d) &
                                 (results['fitness_rule'] == f) &
                                 (results['method'] == m) ]\
                        .fitness.tolist()
            if len(r) == 0:
                r = [-1]
            methods_results.append(r)
        min_eps = deepsig.multi_aso(methods_results, confidence_level=0.95)
        multi_aso_data_list.append({'fitness_rule': f, 'dataset': d, 'min_eps': min_eps})

Model comparisons: 100%|█████████▉| 9980/10000 [00:55<00:00, 184.33it/s]

Model comparisons:   0%|          | 0/10000 [00:00<?, ?it/s]

Model comparisons:   0%|          | 16/10000 [00:00<01:02, 159.69it/s]

Model comparisons:   0%|          | 34/10000 [00:00<00:58, 170.22it/s]

Model comparisons:   1%|          | 52/10000 [00:00<00:58, 171.12it/s]

Model comparisons:   1%|          | 70/10000 [00:00<00:57, 173.11it/s]

Model comparisons:   1%|          | 88/10000 [00:00<00:56, 175.38it/s]

Model comparisons:   1%|          | 106/10000 [00:00<00:56, 175.39it/s]

Model comparisons:   1%|          | 124/10000 [00:00<00:57, 172.28it/s]

Model comparisons:   1%|▏         | 142/10000 [00:00<00:56, 174.16it/s]

Model comparisons:   2%|▏         | 160/10000 [00:00<00:56, 175.34it/s]

Model comparisons:   2%|▏         | 178/10000 [00:01<00:56, 174.84it/s]

Model comparisons:   2%|▏         | 196/10000 [00:01<00:56, 174.82it/s]

Model comparisons:   2%|▏         | 214/10000 [00:01<00:55, 175.14

In [25]:
for aso_result in sorted(multi_aso_data_list, key=lambda x: x['dataset']):
    fitness_rule = aso_result['fitness_rule']
    dataset = aso_result['dataset']
    metrics = fitness_rules_target_metrics[fitness_rule]
    subset_results = results.loc[ (results.fitness_rule == fitness_rule) \
                                      & (results.dataset == dataset) ]

    grouped_results = subset_results\
        .groupby(['method'])\
        .agg({'fitness': ['mean', 'std','count'], metrics['performance']: ['mean', 'std'], metrics['fairness']: ['mean', 'std']})\
        .sort_values(by=('fitness','mean'), ascending=False)

    aso_df = pd.DataFrame(aso_result['min_eps'], columns=methods)
    aso_df['methods'] = methods
    aso_df = aso_df.set_index('methods')

    print('%s | max(%s - %s)' % (dataset, metrics['performance'], metrics['fairness']))
    display(grouped_results)
    print('Significance Testing')
    display(aso_df)

Adult Income | max(Mathew Correlation - Statistical Parity)


fitness                 Mathew Correlation  \
                             mean       std count               mean   
method                                                                 
Fair Transition Loss     0.492021  0.016387    16           0.512027   
Prejudice Remover        0.490575  0.008865    16           0.500024   
Adversarial Debiasing    0.477586  0.014882    16           0.501430   
Standard MLP (baseline)  0.395355  0.012556    17           0.580617   
Gerry Fair Classifier    0.304991  0.128019    16           0.453065   

                                  Statistical Parity            
                              std               mean       std  
method                                                          
Fair Transition Loss     0.010737           0.020006  0.010913  
Prejudice Remover        0.008227           0.009449  0.009857  
Adversarial Debiasing    0.020095           0.023844  0.022968  
Standard MLP (baseline)  0.008613           0.185263  0.010419  
Gerry Fair Classifier    0.173689           0.148074  0.080770

Significance Testing


,Standard MLP (baseline),Fair Transition Loss,Adversarial Debiasing,Prejudice Remover,Gerry Fair Classifier
methods,,,,,
Standard MLP (baseline),1.000000,0.998871,0.998038,0.995876,0.0
Fair Transition Loss,0.004611,1.000000,0.150087,1.000000,0.0
Adversarial Debiasing,0.005003,1.000000,1.000000,1.000000,0.0
Prejudice Remover,0.005958,1.000000,0.122914,1.000000,0.0
Gerry Fair Classifier,1.000000,1.000000,1.000000,1.000000,1.0


Adult Income | max(Mathew Correlation - Equalized Odds)


...

,Standard MLP (baseline),Fair Transition Loss,Adversarial Debiasing,Prejudice Remover,Gerry Fair Classifier
methods,,,,,
Standard MLP (baseline),1.000000,1.000000,0.345320,1.000000,1.0
Fair Transition Loss,1.000000,1.000000,0.367101,1.000000,1.0
Adversarial Debiasing,1.000000,1.000000,1.000000,1.000000,1.0
Prejudice Remover,0.901395,0.984930,0.345980,1.000000,1.0
Gerry Fair Classifier,0.516094,0.506727,0.166599,0.497288,1.0


bank_dataset_reader | max(Accuracy - Equal Opportunity)


fitness                  Accuracy            \
                             mean       std count      mean       std   
method                                                                  
Standard MLP (baseline)  0.803381  0.063751    13  0.902869  0.004330   
Gerry Fair Classifier    0.797935  0.059894    13  0.895994  0.002939   
Fair Transition Loss     0.794587  0.106570    13  0.892878  0.009893   
Prejudice Remover        0.789866  0.075403    13  0.900358  0.002738   
Adversarial Debiasing    0.767442  0.080766    13  0.899892  0.003145   

                        Equal Opportunity            
                                     mean       std  
method                                               
Standard MLP (baseline)          0.099488  0.065112  
Gerry Fair Classifier            0.098058  0.060723  
Fair Transition Loss             0.098291  0.106150  
Prejudice Remover                0.110492  0.076900  
Adversarial Debiasing            0.132450  0.080342

Significance Testing


,Standard MLP (baseline),Fair Transition Loss,Adversarial Debiasing,Prejudice Remover,Gerry Fair Classifier
methods,,,,,
Standard MLP (baseline),1.0,0.69365,0.394565,0.723213,0.994011
Fair Transition Loss,1.0,1.00000,0.663861,1.000000,1.000000
Adversarial Debiasing,1.0,1.00000,1.000000,1.000000,1.000000
Prejudice Remover,1.0,1.00000,0.645733,1.000000,1.000000
Gerry Fair Classifier,1.0,0.75656,0.435725,0.746219,1.000000
